# 데이터 사전

[독립변수]
    
{  
- 공통사항
- subject_id: 사용자 ID
- timestamp: 타임스탬프  
}
- mACSStatus   
  m_charging: 충전여부 (0, 1)  
  0: 비충전  
  1: 충전중  

- mActivity  
  m_activity: 활동 유형 (0~8)  
   0: IN_VEHICLE - 차량 내에 있음 (자동차, 버스 등 교통수단을 이용 중인 상태)  
   1: ON_BICYCLE - 자전거 타는 중 (자전거를 타고 이동 중인 상태)  
   2: ON_FOOT - 걷는 중 (보행 중인 상태로, 걷기와 달리기를 포함하는 일반적인 활동)  
   3: STILL - 정지 상태 (움직임이 없는 상태, 앉아있거나 서있는 등)  
   4: UNKNOWN - 알 수 없음 (활동을 식별할 수 없는 상태)  
   5: TILTING - 기울어짐 (기기가 기울어지거나 방향이 변하는 상태)  
   7: WALKING - 걷는 중 (보행 중인 상태, ON_FOOT보다 더 구체적인 걷기 활동)  
   8: RUNNING - 달리는 중 (달리기 활동을 하는 상태)  

- m_Ambience  
  m_ambience: 주변 소리
  - 주변 소리 레이블과 각 레이블의 확률 목록

- mBle  
  mble: 블루투스 장치
  - 블루투스 장치 주소, 디바이스 분류, rssi 목록

- mGps  
  m_gps: 위치 정보
  - (고도, 위도, 경도, 속도)

- mLight  
  m_light
  - 주변광 (lx 단위)

- mScreenStatus  
  m_screen_use: 화면 사용 여부  
   0: 비사용,   
   1: 화면 사용 중  

- mUsageStats  
  m_usage_stats: 앱 사용
  - 앱 이름과 해당 사용 시간(밀리초 단위)

- mWifi  
  m_wifi
  - 객체 기지국 ID(bssid) 및 rssi 목록

- wHr  
  heart_rate
  - 심박수 기록

- wLight   
  w_light  
  - 주변광 (lx 단위)  

- wPedo  
  burned_calories: 소모 칼로리
  - 칼로리 수
  distance: 거리
  - 거리(미터)
  speed: 속도
  - 속도(km/h)
  step: 걸음
  - 걸음 수
  step_frequency: 보속
  - 분당 걸음 속도

[종속변수]
- subject_id: 사용자 ID  
- sleep_date: 수면 기록 날짜  
- lifelog_date: 일일 활동 날짜 (sleep_date 전날)  
- Q1: 기상 직후 전반적인 수면 품질  
   0: 개인 평균 미만,  
   1: 개인 평균 이상
- Q2: 수면 직전 피로도  
   0: 높은 수준의 피로도,  
   1: 낮은 수준의 피로도
- Q3: 수면 직전 스트레스 수준  
   0: 높은 수준의 스트레스,  
   1: 낮은 수준의 스트레스
- S1: 총 수면 시간 지침 준수 여부(TST)  
   0: 비적절,  
   1: 적절할 수 있음,  
   2: 적절  
- S2: 수면 효율성 지침 준수 여부(SE)  
   0: 부적절  
   1: 적절   
- S3: 수면 시작 지연 시간(SOL, SL)  
   0: 부적절  
   1: 적절  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import glob
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import ast
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from scipy.stats import entropy
from scipy.stats import iqr
import collections

from tqdm import tqdm
from collections import defaultdict
from collections import Counter
from itertools import combinations

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from pytorch_tabnet.tab_network import TabNetEncoder

In [3]:
seed = 810

def set_seed(sd):
    random.seed(sd)
    np.random.seed(sd)
    os.environ['PYTHONHASHSEED'] = str(sd)
    random.seed(sd)
    np.random.seed(sd)
    torch.manual_seed(sd)
    torch.cuda.manual_seed_all(sd)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(seed)

# 데이터 불러오기 및 EDA

In [4]:
data_dir = '/content/drive/MyDrive/1데이콘/ETRI_휴먼이해_인공지능/ETRI_lifelog_dataset/ch2025_data_items'

In [5]:
dataframe = {}
for file_name in sorted(os.listdir(data_dir)):
  if not file_name.endswith('.parquet'):
    continue
  sensor_name = file_name.replace('ch2025_', '').replace('.parquet', '')

  df = pd.read_parquet(os.path.join(data_dir, file_name))
  dataframe[sensor_name] = df
  print(f'Loaded {sensor_name}, shape = {dataframe[sensor_name].shape}')

Loaded mACStatus, shape = (939896, 3)
Loaded mActivity, shape = (961062, 3)
Loaded mAmbience, shape = (476577, 3)
Loaded mBle, shape = (21830, 3)
Loaded mGps, shape = (800611, 3)
Loaded mLight, shape = (96258, 3)
Loaded mScreenStatus, shape = (939653, 3)
Loaded mUsageStats, shape = (45197, 3)
Loaded mWifi, shape = (76336, 3)
Loaded wHr, shape = (382918, 3)
Loaded wLight, shape = (633741, 3)
Loaded wPedo, shape = (748100, 9)


In [6]:
# parquet_files = glob.glob(os.path.join(data_dir, 'ch2025_*.parquet'))

# dataframe = {}

# for file_path in parquet_files:
#     sensor_name = os.path.basename(file_path).replace('ch2025_', '').replace('.parquet', '')
#     dataframe[sensor_name] = pd.read_parquet(file_path)
#     print(f" Loaded: {sensor_name}, shape = {dataframe[sensor_name].shape}")

In [7]:
for name, df in dataframe.items():
  print(f'== {name} ==')
  print(df.head(3), '\n')
  print(df.info(),'\n')
  print('-' * 50)

== mACStatus ==
  subject_id           timestamp  m_charging
0       id01 2024-06-26 12:03:00           0
1       id01 2024-06-26 12:04:00           0
2       id01 2024-06-26 12:05:00           0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939896 entries, 0 to 939895
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   subject_id  939896 non-null  object        
 1   timestamp   939896 non-null  datetime64[ns]
 2   m_charging  939896 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 21.5+ MB
None 

--------------------------------------------------
== mActivity ==
  subject_id           timestamp  m_activity
0       id01 2024-06-26 12:03:00           4
1       id01 2024-06-26 12:04:00           0
2       id01 2024-06-26 12:05:00           0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961062 entries, 0 to 961061
Data columns (total 3 columns):
 #   Column

In [8]:
# 결측치 및 요약
for name, df in dataframe.items():
  print(f'== {name} ==')
  print(f'Missing Value', (df.isnull().mean() * 100).round(2).to_dict())
  num_cols = df.select_dtypes(include='number').columns
  if len(num_cols) > 0:
    print(f'Summary Statistic', '\n', df[num_cols].describe().loc[['mean','std','min','max']])
  print()

== mACStatus ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_charging': 0.0}
Summary Statistic 
       m_charging
mean    0.291378
std     0.454397
min     0.000000
max     1.000000

== mActivity ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_activity': 0.0}
Summary Statistic 
       m_activity
mean    2.986991
std     1.132508
min     0.000000
max     8.000000

== mAmbience ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_ambience': 0.0}

== mBle ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_ble': 0.0}

== mGps ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_gps': 0.0}

== mLight ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_light': 0.0}
Summary Statistic 
             m_light
mean     185.176751
std     1939.970572
min        0.000000
max   334306.000000

== mScreenStatus ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_screen_use': 0.0}
Summary Statistic 
       m_screen_use
mean      0.228389
std       0.41979

In [9]:
for name, df in dataframe.items():
  if name.endswith('Light'):
    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      print(col)
      q1 = df[col].quantile(0.25)
      q3 = df[col].quantile(0.75)
      iqr = q3 - q1
      lower = q1 - (1.5 * iqr)
      upper = q3 + (1.5 * iqr)
      outliers = df[(df[col] < lower) | (df[col] > upper)]
      if not outliers.empty:
        print(f'Lower: {lower}, Upper: {upper}, Outlier length: {len(outliers)}, Outlier ratio: {(len(outliers) / len(df)) * 100:.2f}%, \n')
      df.loc[df[col] > upper, col] = 100000.0
  #   if len(col) > 0:
  #     sns.boxplot(y=df[col])
  #     plt.title(f'Boxplot of {name}')
  #     plt.show()
  # print()

m_light
Lower: -235.5, Upper: 392.5, Outlier length: 11638, Outlier ratio: 12.09%, 

w_light
Lower: -198.0, Upper: 330.0, Outlier length: 71751, Outlier ratio: 11.32%, 



In [10]:
# 초 단위 제거
for name, df in dataframe.items():
  df['timestamp'] =  df['timestamp'].dt.floor('min')
  df['date'] = df['timestamp'].dt.date
  dataframe[name] = df

In [11]:
# 변수마다 데이터프레임화
for key, df in dataframe.items():
  globals()[f'{key}_df'] = df

In [12]:
# dataframe = {
#     'mACStatus': (mACStatus_df, 'timestamp'),
#     'mActivity': (mActivity_df, 'timestamp'),
#     'mAmbience': (mAmbience_df, 'timestamp'),
#     'mBle': (mBle_df, 'timestamp'),
#     'mGps': (mGps_df, 'timestamp'),
#     'mLight': (mLight_df, 'timestamp'),
#     'mScreenStatus': (mScreenStatus_df, 'timestamp'),
#     'mUsageStats': (mUsageStats_df, 'timestamp'),
#     'mWifi': (mWifi_df, 'timestamp'),
#     'wHr': (wHr_df, 'timestamp'),
#     'wLight': (wLight_df, 'timestamp'),
#     'wPedo': (wPedo_df, 'timestamp'),
# }

# 변수별 전처리

In [13]:
# def explode(df):
#   col = [col for col in df.columns if col.startswith('m_')]
#   assert len(col) == 1
#   col = col[0]

#   explode_df = df.explode(col, ignore_index=True)
#   detail_df = pd.json_normalize(explode_df[col])
#   result_df = pd.concat([explode_df.drop(columns = col), detail_df], axis=1)

#   return result_df

In [14]:
def sleep_lifelog(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['sleep_date'] = np.where(
        df['timestamp'].dt.hour < 7,
        df['timestamp'].dt.date,
        df['timestamp'].dt.date + pd.Timedelta(days=1)
    )
    df['lifelog_date'] = pd.to_datetime(df['sleep_date']) - pd.Timedelta(days=1)

    return df

In [15]:
def find_col(df):
    col = [col for col in df.columns if col.startswith('m_') or col.startswith('w_') or col.startswith('he')]
    assert len(col) == 1
    col = col[0]

    return col

In [16]:
def cut_time(hour):
    if 22 <= hour or 0 <= hour < 7:
        return 'Q1'
    elif 7 <= hour < 22:
        return 'Q2_Q3'

In [17]:
time_list = ['step_list']

def view_corr(df):
    for tl in time_list:
      numeric_cols = df.select_dtypes(include='number').columns
      tl_cols = [col for col in numeric_cols if col.startswith(tl)]
      corr_df = df[tl_cols]
      print(f'{tl}_corr_list')
      print(pd.DataFrame(corr_df.corr().drop_duplicates()))
      print('-'*50, '\n')

### mACStatus

In [18]:
def preprocessing_mACStatus(dfs):
    df = sleep_lifelog(dfs)

    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)

    col = find_col(df)

    results = []
    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):
        status = part_group[col].values  # 0/1 상태

        # 충전상태 길이
        charging_period = sum(status) / len(status)
        row[f'{part}@charging_period'] = charging_period

        # 충전 상태 평균
        # ratio_charging = status.mean()
        # row[f'{part}@charging_ratio'] = ratio_charging

        # 상태 전이 횟수
        transitions = (status[1:] != status[:-1]).sum() / len(status)
        row[f'{part}@charging_transitions'] = transitions

        # 충전 패턴 분포
        _, counts = np.unique(status, return_counts=True)
        prob = counts / counts.sum()
        ent = entropy(prob) # 1에 가까울수록 충전 상태가 자주 바뀜
        row[f'{part}@charging_entropy'] = ent

        # 연속된 1 상태 길이
        length = []
        current_len = 0
        for val in status:
            if val == 1:
                current_len += 1
            elif current_len > 0:
                length.append(current_len)
                current_len = 0
        if current_len > 0:
            length.append(current_len)

        # 평균 충전기간
        avg_charging_period = (np.mean(length) / len(length)) if length else 0
        row[f'{part}@charging_avg_period'] = avg_charging_period

        # 최대 충전기간
        max_charging_period = (np.max(length)/ len(length)) if length else 0
        row[f'{part}@charging_max_period'] = max_charging_period

        # 불안전성
        row[f'{part}@charging_instability'] = (1 + ent) * (1 + transitions) * (1 + charging_period)

      # 결과값 데이터프레임
      results.append(row)

    return pd.DataFrame(results).fillna(0)

pre_mACStatus_df = preprocessing_mACStatus(mACStatus_df)

def cummean_mACStatus(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_mACStatus_df = cummean_mACStatus(pre_mACStatus_df)

def final_mACStatus(pre_mACStatus_df, cummean_mACStatus_df):
  merge_df = pre_mACStatus_df.merge(cummean_mACStatus_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_mACStatus_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'S1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'S2_S3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_mACStatus_df = final_mACStatus(pre_mACStatus_df, cummean_mACStatus_df)

Q1@charging_period_cummean
Q1@charging_transitions_cummean
Q1@charging_entropy_cummean
Q1@charging_avg_period_cummean
Q1@charging_max_period_cummean
Q1@charging_instability_cummean
Q2_Q3@charging_period_cummean
Q2_Q3@charging_transitions_cummean
Q2_Q3@charging_entropy_cummean
Q2_Q3@charging_avg_period_cummean
Q2_Q3@charging_max_period_cummean
Q2_Q3@charging_instability_cummean


In [19]:
final_mACStatus_df

,subject_id,date
0,id01,2024-06-26
1,id01,2024-06-27
2,id01,2024-06-28
3,id01,2024-06-29
4,id01,2024-06-30
...,...,...
798,id10,2024-09-22
799,id10,2024-09-23
800,id10,2024-09-24
801,id10,2024-09-25


### mActivity

In [20]:
def preprocessing_mActivity(dfs):
  df = sleep_lifelog(dfs)

  df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
  df['date'] = df['lifelog_date'].dt.date

  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df = df.sort_values(['subject_id', 'timestamp'])
  df['time_part'] = df['timestamp'].dt.hour.map(cut_time)

  col = find_col(df)

  results = []
  for (sub, date), daily_group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':sub, 'date':date}
      for part, part_group in daily_group.groupby('time_part'):
        status = part_group[col].values

        # 활동 다양성
        activity_diversity = (part_group[col].nunique() / 8)
        row[f'{part}@activity_diversity'] = activity_diversity
        # 가장 많은 활동
        # activity_frequence = part_group[col].mode()[0] if not part_group[col].mode().empty else np.nan
        # row[f'{part}@activity_frequence'] = activity_frequence

        # 활동 유형 비율
        count = part_group[col].value_counts(normalize=True)

        # 가장 많은 활동 비율
        # row[f'{part}@activity_frequence_ratio'] = count.max()

        # 활동 다양성 분포
        ent = entropy(count)
        row[f'{part}@activity_entropy'] = ent

        # 활동 바뀐 횟수
        changes = (status[1:] != status[:-1]).sum()
        row[f'{part}@activity_change'] = changes / len(status)

        # 연속된 활동 저장
        length_activity = {}
        for j in range(9):
          length = []
          current = 0
          for val in status:
            if val == j:
              current += 1
            elif current > 0:
              length.append(current)
              current = 0
          if current > 0:
            length.append(current)
          length_activity[j] = length

        # 총 활동 수
        row[f'{part}@total_activity'] = sum(k + 1 * item
                                            for k, lst in length_activity.items()
                                            if k not in [3, 4]
                                            for item in lst) / len(length_activity)

        # # 활동 유형별 비율
        dynamic_state = [0,1,2,7,8]
        static_state = [3,5]

        # dynamic_state_cnt = np.isin(status, dynamic_state).sum()
        # row[f'{part}@dynamic_state_cnt'] = dynamic_state_cnt
        dynamic_state_ratio = sum([count.get(i,0) for i in dynamic_state])
        row[f'{part}@dynamic_state_ratio'] = dynamic_state_ratio

        # static_state_cnt = np.isin(status, static_state).sum()
        # row[f'{part}@static_state_cnt'] = static_state_cnt
        static_state_ratio = sum([count.get(i,0) for i in static_state])
        row[f'{part}@static_state_ratio'] = static_state_ratio

        # 활동 불안전성
        row[f'{part}@activity_instability'] = (1 + ent) * (1 + changes) * (1 + activity_diversity)

      results.append(row)

  return pd.DataFrame(results)

pre_mActivity_df = preprocessing_mActivity(mActivity_df)

def cummean_mActivity(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_mActivity_df = cummean_mActivity(pre_mActivity_df)

def final_mActivity(pre_mActivity_df, cummean_mActivity_df):
  merge_df = pre_mActivity_df.merge(cummean_mActivity_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_mActivity_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'Q1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'Q2_Q3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_mActivity_df = final_mActivity(pre_mActivity_df, cummean_mActivity_df)

Q1@activity_diversity_cummean
Q1@activity_entropy_cummean
Q1@activity_change_cummean
Q1@total_activity_cummean
Q1@dynamic_state_ratio_cummean
Q1@static_state_ratio_cummean
Q1@activity_instability_cummean
Q2_Q3@activity_diversity_cummean
Q2_Q3@activity_entropy_cummean
Q2_Q3@activity_change_cummean
Q2_Q3@total_activity_cummean
Q2_Q3@dynamic_state_ratio_cummean
Q2_Q3@static_state_ratio_cummean
Q2_Q3@activity_instability_cummean


In [21]:
final_mActivity_df

,subject_id,date,Q1@activity_diversity,Q1@activity_entropy,Q1@activity_change,Q1@total_activity,Q1@dynamic_state_ratio,Q1@static_state_ratio,Q1@activity_instability,Q2_Q3@activity_diversity,Q2_Q3@activity_entropy,Q2_Q3@activity_change,Q2_Q3@total_activity,Q2_Q3@dynamic_state_ratio,Q2_Q3@static_state_ratio,Q2_Q3@activity_instability
0,id01,2024-06-26,1,1,1,1,1,0,1,0,0,0,1,0,1,0
1,id01,2024-06-27,1,1,1,1,1,0,1,1,1,0,1,1,0,1
2,id01,2024-06-28,1,0,0,0,0,1,0,0,0,0,0,0,0,0
3,id01,2024-06-29,0,0,0,0,0,1,0,1,1,1,1,1,0,1
4,id01,2024-06-30,1,1,1,1,1,0,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,id10,2024-09-22,1,0,1,0,0,1,0,0,0,0,0,0,0,0
799,id10,2024-09-23,0,0,0,0,0,1,0,0,0,0,0,0,0,0
800,id10,2024-09-24,0,0,0,0,0,1,0,0,0,0,0,0,1,0
801,id10,2024-09-25,0,0,0,0,0,1,0,0,0,0,0,0,1,0


### mAmbience

In [22]:
def preprocessing_mAmbience(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': subj, 'date': date}
      for part, part_group in group.groupby('time_part'):
        all_labels_scores = []

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device

          sum_score = sum(float(score) for _, score in flat if isinstance(score, (int, float, str)))
          all_labels_scores.extend([
              (label, float(normalized_score))
              for label, score in flat
              if sum_score > 0
              for normalized_score in [float(score) / sum_score]
              if 0 < float(normalized_score) < 1
              ])

          if all_labels_scores:
            label_list = [label for label, _ in all_labels_scores]
            score_list = [score for _, score in all_labels_scores]
            labels, scores = zip(*all_labels_scores)
            scores = list(scores)
            max_idx = np.argmax(scores)
            min_idx = np.argmin(scores)

            # 소음 최대
            # row[f'{part}@ambience_max_label'] = labels[max_idx]

            # 소음 최소
            # row[f'{part}@ambience_min_label'] = labels[min_idx]

            range = scores[max_idx] - scores[min_idx]
            row[f'{part}@ambience_range'] = range

            # 소음 형태 고유값
            ambience_cnt = len(set(label_list)) / len(label_list)
            row[f'{part}@ambience_cnt'] = ambience_cnt

            # 소음 형태 다양성 분포
            probs_counter = Counter(label_list)
            total = sum(probs_counter.values())
            probs = [v / total for v in probs_counter.values()]
            ent = entropy(probs)
            row[f'{part}@ambience_entropy'] = ent

            # 기초 통계량
            mean = np.mean(scores) if sum(scores) > 0 else 0
            row[f'{part}@ambience_mean'] = mean
            std = np.std(scores) if sum(scores) > 0 else 0
            row[f'{part}@ambience_std'] = std
            cov = (np.std(scores) / np.mean(scores)) if sum(scores) > 0 else 0
            row[f'{part}@ambience_cov'] = cov

            # 소음 불안전성
            row[f'{part}@ambience_instability'] = (1 + ent) * (1 + ambience_cnt) * (1 + cov)

      results.append(row)

    return pd.DataFrame(results)

pre_mAmbience_df = preprocessing_mAmbience(mAmbience_df)

def cummean_mAmbience(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_mAmbience_df = cummean_mAmbience(pre_mAmbience_df)

def final_mAmbience(pre_mAmbience_df, cummean_mAmbience_df):
  merge_df = pre_mAmbience_df.merge(cummean_mAmbience_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_mAmbience_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'Q1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'Q2_Q3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_mAmbience_df = final_mAmbience(pre_mAmbience_df, cummean_mAmbience_df)

Q1@ambience_range_cummean
Q1@ambience_cnt_cummean
Q1@ambience_entropy_cummean
Q1@ambience_mean_cummean
Q1@ambience_std_cummean
Q1@ambience_cov_cummean
Q1@ambience_instability_cummean
Q2_Q3@ambience_range_cummean
Q2_Q3@ambience_cnt_cummean
Q2_Q3@ambience_entropy_cummean
Q2_Q3@ambience_mean_cummean
Q2_Q3@ambience_std_cummean
Q2_Q3@ambience_cov_cummean
Q2_Q3@ambience_instability_cummean


In [23]:
final_mAmbience_df

,subject_id,date,Q1@ambience_range,Q1@ambience_cnt,Q1@ambience_entropy,Q1@ambience_mean,Q1@ambience_std,Q1@ambience_cov,Q1@ambience_instability,Q2_Q3@ambience_range,Q2_Q3@ambience_cnt,Q2_Q3@ambience_entropy,Q2_Q3@ambience_mean,Q2_Q3@ambience_std,Q2_Q3@ambience_cov,Q2_Q3@ambience_instability
0,id01,2024-06-26,1,1,0,1,1,1,1,0,0,0,1,1,1,0
1,id01,2024-06-27,1,0,0,1,1,1,0,0,1,1,0,0,0,1
2,id01,2024-06-28,1,0,0,1,1,1,0,0,1,1,0,0,0,1
3,id01,2024-06-29,1,0,0,1,1,1,0,0,1,1,0,0,0,1
4,id01,2024-06-30,1,0,0,1,1,1,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,id10,2024-09-22,0,1,1,0,0,1,1,0,0,0,1,1,1,0
799,id10,2024-09-23,0,0,1,0,0,0,0,0,0,0,0,0,0,0
800,id10,2024-09-24,0,1,1,0,0,0,1,0,0,0,1,1,1,0
801,id10,2024-09-25,0,1,1,0,0,0,1,1,0,0,1,1,1,0


### mBle

In [24]:
device_class_labels = {'Unknown': ['0', '7936'],
                      'Computer': ['256', '284'],
                      'Phone': ['524'],
                      'Audio_Video':['1024', '1028', '1044', '1048', '1060', '1064', '1084'],
                      'Peripheral':['1280', '1344', '1408'],
                      'Wearable':['1796'],
                      'Health':['2324']}

device_labels = {
    'Unknown': device_class_labels['Unknown'],
    'Recognizable': []
}

for k, v in device_class_labels.items():
  if k != 'Unknown':
    device_labels['Recognizable'] += v

print(device_labels)

{'Unknown': ['0', '7936'], 'Recognizable': ['256', '284', '524', '1024', '1028', '1044', '1048', '1060', '1064', '1084', '1280', '1344', '1408', '1796', '2324']}


In [25]:
def preprocessing_mBle(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):

        rssi_list = []
        address_list = []
        class_counts = {f'class_{label}_cnt': 0 for label in device_labels}

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          for device in flat:

              # rssi 목록
              rssi = int(device.get('rssi'))
              rssi_list.append(rssi)

              # address 목록
              address = str(device.get('address'))
              address_list.append(address)

              # 디바이스 장치 수
              device_class = str(device.get('device_class'))
              for label, ids in device_labels.items():
                if device_class in ids:
                  class_counts[f'class_{label}_cnt'] += 1

        # rssi 고유값
        rssi_cnt = len(set(rssi_list)) / len(rssi_list)
        row[f'{part}@rssi_cnt'] = rssi_cnt

        # rssi 다양성
        rssi_counter = Counter(address_list)
        total = sum(rssi_counter.values())
        probs = [v / total for v in rssi_counter.values()]
        ent_rssi = entropy(probs)
        row[f'{part}@rssi_entropy'] = ent_rssi

        # address 고유값
        address_cnt = len(set(address_list)) / len(address_list)
        row[f'{part}@address_cnt'] = address_cnt

        # address 다양성
        probs_counter = Counter(address_list)
        total = sum(probs_counter.values())
        probs = [v / total for v in probs_counter.values()]
        ent_address = entropy(probs)
        row[f'{part}@address_entropy'] = ent_address

        # 라벨 수
        total_count = sum(class_counts.values())
        for label in device_labels:
          if total_count > 0:
            row[f'{part}@{label}_cnt'] = class_counts[f'class_{label}_cnt'] / total_count
          else:
            row[f'{part}@{label}_cnt'] = 0
          # row[f'{part}@{label}_ratio'] = (class_counts[f'class_{label}_cnt'] / total_devices if total_devices > 0 else 0)

        # 라벨 다양성
        counts = np.array(list(class_counts.values()))
        ent_label = entropy(counts)
        row[f'{part}@label_entropy'] = ent_label

        # 기초 통계
        mean = np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_mean'] = np.mean(rssi_list) if rssi_list else 0
        min = np.min(rssi_list) if rssi_list else 0
        max = np.max(rssi_list) if rssi_list else 0
        range = np.max(rssi_list) - np.min(rssi_list)
        row[f'{part}@rssi_range'] = range
        std = np.std(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_std'] = std
        cov = np.std(rssi_list) / np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_cov'] = cov

        # mble 불안전성
        row[f'{part}@mble_instability'] = ((1 + ent_rssi) * (1 + rssi_cnt) *
         (1 + ent_address) * (1 + address_cnt) *
          (1 + ent_label) * (1 + (class_counts['class_Recognizable_cnt'] if class_counts['class_Recognizable_cnt'] else 0)) *
           (1 + cov))

      results.append(row)

    return pd.DataFrame(results)

pre_mBle_df = preprocessing_mBle(mBle_df)

def cummean_mBle(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_mBle_df = cummean_mBle(pre_mBle_df)

def final_mBle(pre_mBle_df, cummean_mBle_df):
  merge_df = pre_mBle_df.merge(cummean_mBle_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_mBle_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'Q1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'Q2_Q3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_mBle_df = final_mBle(pre_mBle_df, cummean_mBle_df)

Q1@rssi_cnt_cummean
Q1@rssi_entropy_cummean
Q1@address_cnt_cummean
Q1@address_entropy_cummean
Q1@Unknown_cnt_cummean
Q1@Recognizable_cnt_cummean
Q1@label_entropy_cummean
Q1@rssi_mean_cummean
Q1@rssi_range_cummean
Q1@rssi_std_cummean
Q1@rssi_cov_cummean
Q1@mble_instability_cummean
Q2_Q3@rssi_cnt_cummean
Q2_Q3@rssi_entropy_cummean
Q2_Q3@address_cnt_cummean
Q2_Q3@address_entropy_cummean
Q2_Q3@Unknown_cnt_cummean
Q2_Q3@Recognizable_cnt_cummean
Q2_Q3@label_entropy_cummean
Q2_Q3@rssi_mean_cummean
Q2_Q3@rssi_range_cummean
Q2_Q3@rssi_std_cummean
Q2_Q3@rssi_cov_cummean
Q2_Q3@mble_instability_cummean


In [26]:
final_mBle_df

,subject_id,date,Q1@rssi_cnt,Q1@rssi_entropy,Q1@address_cnt,Q1@address_entropy,Q1@Unknown_cnt,Q1@Recognizable_cnt,Q1@label_entropy,Q1@rssi_mean,Q1@rssi_range,Q1@rssi_std,Q1@rssi_cov,Q1@mble_instability,Q2_Q3@rssi_cnt,Q2_Q3@rssi_entropy,Q2_Q3@address_cnt,Q2_Q3@address_entropy,Q2_Q3@Unknown_cnt,Q2_Q3@Recognizable_cnt,Q2_Q3@label_entropy,Q2_Q3@rssi_mean,Q2_Q3@rssi_range,Q2_Q3@rssi_std,Q2_Q3@rssi_cov,Q2_Q3@mble_instability
0,id01,2024-06-26,0,1,0,1,0,1,1,1,1,1,0,1,1,0,0,0,0,1,0,1,0,1,0,0
1,id01,2024-06-27,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,0,0,1,0,0,0,1,1,0
2,id01,2024-06-28,0,1,0,1,0,1,1,1,1,1,0,1,1,0,0,0,1,0,0,1,1,1,0,0
3,id01,2024-06-29,1,0,1,0,1,0,0,1,0,0,1,0,0,1,1,1,1,0,0,0,1,0,1,0
4,id01,2024-06-30,0,1,1,1,0,1,1,0,1,1,1,1,0,1,0,1,1,0,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,id10,2024-09-22,0,0,0,0,1,0,0,0,0,0,1,0,1,1,1,1,0,1,1,1,0,1,0,1
722,id10,2024-09-23,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
723,id10,2024-09-24,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0
724,id10,2024-09-25,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,1,1


### mGps

In [27]:
from math import radians, sin, cos, sqrt, atan2

def haversine(coord1, coord2):
    R = 6371  # 지구 반지름 (km)
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [28]:
def preprocessing_mGps(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):
        distances = []
        altitudes_list = []
        latitudes_list = []
        longitudes_list = []
        speeds_list = []
        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          for entry in flat:
            altitudes_list.append(float(entry['altitude']))
            latitudes_list.append(float(entry['latitude']))
            longitudes_list.append(float(entry['longitude']))
            speeds_list.append(float(entry['speed']))

        # 중심점
        if latitudes_list and longitudes_list:
          center = (np.mean(latitudes_list), np.mean(longitudes_list))
          distances = [haversine(center, (lat, lng)) for lat, lng in zip(latitudes_list, longitudes_list)]

          # 중심점으로부터 최대 이동거리
          radius = max(distances) if distances else np.nan
          row[f'{part}@center_from_max_distance'] = radius

          # 중심점으로부터 총 이동거리
          center_total_distance = sum(distances)
          row[f'{part}@center_total_distance'] = center_total_distance

          coords = list(zip(latitudes_list, longitudes_list))
          if len(coords) >= 2:
            total_distance = sum(haversine(coords[i], coords[i+1]) for i in range(len(coords) - 1))
          else:
            total_distance = 0.0
          row[f'{part}@total_distance'] = total_distance

          # 이동경로
          path_tortuosity = ((center_total_distance / total_distance) if total_distance > 0 else 0)
          row[f'{part}@path_tortuosity'] = path_tortuosity

        # 고도 기초 통계량
        altitude_mean = np.mean(altitudes_list) if altitudes_list else 0
        row[f'{part}@altitude_mean'] = altitude_mean
        altitude_std = np.std(altitudes_list) if altitudes_list else 0
        row[f'{part}@altitude_std'] = altitude_std
        altitude_max = np.max(altitudes_list) if altitudes_list else 0
        # row[f'{part}@altitude_max'] = altitude_max
        altitude_min = np.min(altitudes_list) if altitudes_list else 0
        # row[f'{part}@altitude_min'] = altitude_min
        altitude_range = altitude_max - altitude_min
        row[f'{part}@altitude_range'] = altitude_range
        altitude_cov = np.std(altitudes_list) / np.mean(altitudes_list) if altitudes_list else 0
        row[f'{part}@altitude_cov'] = altitude_cov

        # 고도 다양성
        hist_alt, _ = np.histogram(altitudes_list, bins=10, density=True)
        alt_ent = entropy(hist_alt) if hist_alt.sum() > 0 else 0
        row[f'{part}@altitude_entropy'] = alt_ent

        # 속도 기초 통계량
        speed_mean = np.mean(speeds_list) if speeds_list else 0
        row[f'{part}@speed_mean'] = speed_mean
        speed_max = np.max(speeds_list) if speeds_list else 0
        row[f'{part}@speed_max'] = speed_max
        speed_std = np.std(speeds_list) if speeds_list else 0
        row[f'{part}@speed_std'] = speed_std
        speed_cov =   np.std(speeds_list) / np.mean(speeds_list) if speeds_list else 0
        row[f'{part}@speed_cov'] = speed_cov

        # 정지 속도 수
        row[f'{part}@speed_zero_count'] =  np.sum([s == 0 for s in speeds_list]) / len(speeds_list) if speeds_list else 0

        # 속도 다양성
        hist_spd, _ = np.histogram(speeds_list, bins=10, density=True)
        speed_ent = entropy(hist_spd) if hist_spd.sum() > 0 else 0
        row[f'{part}@speed_entropy'] = speed_ent

        # 위도 경도 요약
        longitude_std = np.std(longitudes_list) if longitudes_list else 0
        row[f'{part}@longitude_std'] = longitude_std
        latitude_std = np.std(latitudes_list) if latitudes_list else 0
        row[f'{part}@latitude_std'] = latitude_std

        # mgps 불안전성
        row[f'{part}@mgps_instability'] = ((1 + speed_ent) * (1+ alt_ent) * (1 + abs(1 - path_tortuosity)) *
         (1 + speed_cov) * (1 + altitude_cov))

      results.append(row)

    return pd.DataFrame(results)

pre_mGps_df = preprocessing_mGps(mGps_df)

def cummean_mGps(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_mGps_df = cummean_mGps(pre_mGps_df)

def final_mGps(pre_mGps_df, cummean_mGps_df):
  merge_df = pre_mGps_df.merge(cummean_mGps_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_mGps_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'Q1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'Q2_Q3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_mGps_df = final_mGps(pre_mGps_df, cummean_mGps_df)

Q1@center_from_max_distance_cummean
Q1@center_total_distance_cummean
Q1@total_distance_cummean
Q1@path_tortuosity_cummean
Q1@altitude_mean_cummean
Q1@altitude_std_cummean
Q1@altitude_range_cummean
Q1@altitude_cov_cummean
Q1@altitude_entropy_cummean
Q1@speed_mean_cummean
Q1@speed_max_cummean
Q1@speed_std_cummean
Q1@speed_cov_cummean
Q1@speed_zero_count_cummean
Q1@speed_entropy_cummean
Q1@longitude_std_cummean
Q1@latitude_std_cummean
Q1@mgps_instability_cummean
Q2_Q3@center_from_max_distance_cummean
Q2_Q3@center_total_distance_cummean
Q2_Q3@total_distance_cummean
Q2_Q3@path_tortuosity_cummean
Q2_Q3@altitude_mean_cummean
Q2_Q3@altitude_std_cummean
Q2_Q3@altitude_range_cummean
Q2_Q3@altitude_cov_cummean
Q2_Q3@altitude_entropy_cummean
Q2_Q3@speed_mean_cummean
Q2_Q3@speed_max_cummean
Q2_Q3@speed_std_cummean
Q2_Q3@speed_cov_cummean
Q2_Q3@speed_zero_count_cummean
Q2_Q3@speed_entropy_cummean
Q2_Q3@longitude_std_cummean
Q2_Q3@latitude_std_cummean
Q2_Q3@mgps_instability_cummean


In [29]:
final_mGps_df

,subject_id,date,Q1@center_from_max_distance,Q1@center_total_distance,Q1@total_distance,Q1@path_tortuosity,Q1@altitude_mean,Q1@altitude_std,Q1@altitude_range,Q1@altitude_cov,Q1@altitude_entropy,Q1@speed_mean,Q1@speed_max,Q1@speed_std,Q1@speed_cov,Q1@speed_zero_count,Q1@speed_entropy,Q1@longitude_std,Q1@latitude_std,Q1@mgps_instability,Q2_Q3@center_from_max_distance,Q2_Q3@center_total_distance,Q2_Q3@total_distance,Q2_Q3@path_tortuosity,Q2_Q3@altitude_mean,Q2_Q3@altitude_std,Q2_Q3@altitude_range,Q2_Q3@altitude_cov,Q2_Q3@altitude_entropy,Q2_Q3@speed_mean,Q2_Q3@speed_max,Q2_Q3@speed_std,Q2_Q3@speed_cov,Q2_Q3@speed_zero_count,Q2_Q3@speed_entropy,Q2_Q3@longitude_std,Q2_Q3@latitude_std,Q2_Q3@mgps_instability
0,id01,2024-06-26,1,1,1,1,0,0,0,0,0,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,1,0,1,1,1
1,id01,2024-06-27,1,1,1,1,0,0,0,0,1,0,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0,0,1,1,0,1,1,1
2,id01,2024-06-28,1,1,1,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,1,0,1,1,1,1,1,0,0,0,0,1,1,0,1,1,1
3,id01,2024-06-29,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1
4,id01,2024-06-30,1,1,1,1,1,0,0,0,1,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,0,1,0,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,id10,2024-09-22,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,1,1,1,1,0,0,0,0,0,1,1,0,1
755,id10,2024-09-23,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
756,id10,2024-09-24,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0
757,id10,2024-09-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0


### mLight

In [30]:
def preprocessing_mLight(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (sub, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': sub, 'date': date}
      for time_label, block_group in group.groupby('time_part'):
        light_list = block_group[col].tolist()

        # mlight 기초 통계
        mean = np.mean(light_list)
        row[f'{time_label}@mlight_mean'] = mean
        std = np.std(light_list)
        row[f'{time_label}@mlight_std'] = std
        cov = np.std(light_list) / np.mean(light_list)
        row[f'{time_label}@mlight_cov'] = cov
        max = np.max(light_list)
        # row[f'{time_label}@mlight_max'] = max
        min = np.min(light_list)
        # row[f'{time_label}@mlight_min'] = min
        range = max - min
        row[f'{time_label}@mlight_range'] = range

        # might 고유값
        light_cnt = len(set(light_list)) / len(light_list)
        row[f'{time_label}@mlight_cnt'] = light_cnt

        # mlight 다양성
        hist_light, _ = np.histogram(light_list, bins=10, density=True)
        ent = entropy(hist_light) if hist_light.sum() > 0 else 0
        row[f'{time_label}@mlight_entropy'] = ent

        # mlight 불안전성
        row[f'{time_label}@mlight_instability'] = (1 + ent) * (1 + cov) * (1 + light_cnt)

      results.append(row)

    return pd.DataFrame(results)

pre_mLight_df = preprocessing_mLight(mLight_df)

def cummean_mLight(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_mLight_df = cummean_mLight(pre_mLight_df)

def final_mLight(pre_mLight_df, cummean_mLight_df):
  merge_df = pre_mLight_df.merge(cummean_mLight_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_mLight_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'Q1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'Q2_Q3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_mLight_df = final_mLight(pre_mLight_df, cummean_mLight_df)

Q1@mlight_mean_cummean
Q1@mlight_std_cummean
Q1@mlight_cov_cummean
Q1@mlight_range_cummean
Q1@mlight_cnt_cummean
Q1@mlight_entropy_cummean
Q1@mlight_instability_cummean
Q2_Q3@mlight_mean_cummean
Q2_Q3@mlight_std_cummean
Q2_Q3@mlight_cov_cummean
Q2_Q3@mlight_range_cummean
Q2_Q3@mlight_cnt_cummean
Q2_Q3@mlight_entropy_cummean
Q2_Q3@mlight_instability_cummean


In [31]:
final_mLight_df

,subject_id,date,Q1@mlight_mean,Q1@mlight_std,Q1@mlight_cov,Q1@mlight_range,Q1@mlight_cnt,Q1@mlight_entropy,Q1@mlight_instability,Q2_Q3@mlight_mean,Q2_Q3@mlight_std,Q2_Q3@mlight_cov,Q2_Q3@mlight_range,Q2_Q3@mlight_cnt,Q2_Q3@mlight_entropy,Q2_Q3@mlight_instability
0,id01,2024-06-26,1,1,1,1,1,0,1,0,0,1,0,1,0,1
1,id01,2024-06-27,0,1,1,1,0,0,1,1,0,1,0,0,0,1
2,id01,2024-06-28,0,0,0,0,0,1,0,0,0,1,0,1,0,1
3,id01,2024-06-29,0,0,1,0,0,0,1,1,1,0,0,0,1,0
4,id01,2024-06-30,0,0,1,1,0,0,1,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,id10,2024-09-22,0,0,0,0,1,0,1,1,1,0,0,1,1,0
798,id10,2024-09-23,0,0,1,0,0,0,1,0,0,0,0,0,0,0
799,id10,2024-09-24,0,0,0,0,1,1,1,1,1,0,0,0,1,0
800,id10,2024-09-25,0,0,0,0,1,1,1,0,0,1,0,1,1,1


### mScreenStatus

In [32]:
def preprocessing_mScreenStatus(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []
    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):
        status = part_group[col].values

        # 총 화면 켜진 기간
        # on_screen_period = sum(status)
        # row[f'{part}on_screen_period'] = on_screen_period

        # 화면 켜진상태 비율
        on_screen_ratio = status.mean()
        row[f'{part}@on_screen_ratio'] = on_screen_ratio

        # 상태 전이 횟수
        transitions = (status[1:] != status[:-1]).sum() / len(status)
        row[f'{part}@on_screen_transitions'] = transitions

        # 사용 패턴 분포
        _, counts = np.unique(status, return_counts=True)
        prob = counts / counts.sum()
        ent = entropy(prob) # 1에 가까울수록 사용 상태가 자주 바뀜
        row[f'{part}@on_screen_entropy'] = ent

        # 연속된 1 상태 길이
        length = []
        current_len = 0
        for val in status:
            if val == 1:
                current_len += 1
            elif current_len > 0:
                length.append(current_len)
                current_len = 0
        # 배열이 1로 끝날 때 마지막 구간 길이
        if current_len > 0:
            length.append(current_len)

        # 화면 켜진 기간 기초통계량
        avg_on_screen = np.mean(length) if length else 0
        row[f'{part}@avg_on_screen'] = avg_on_screen
        std_on_screen = np.std(length) if length else 0
        row[f'{part}@std_on_screen'] = std_on_screen
        cov_on_screen = (np.std(length) / np.mean(length)) if length else 0
        row[f'{part}@cov_on_screed'] = cov_on_screen
        max_on_screen = np.max(length) if length else 0
        # row[f'{part}@max_on_screen'] = max_on_screen
        min_on_screen = np.min(length) if length else 0
        # row[f'{part}@min_on_screen'] = min_on_screen
        range_on_screen = max_on_screen - min_on_screen
        row[f'{part}@range_on_screen'] = range_on_screen

        # 화면 불안전성
        row[f'{part}@mscreen_instability'] = (1 + ent) * (1 + transitions) * (1 + on_screen_ratio) * (1+ cov_on_screen)

      results.append(row)

    return pd.DataFrame(results)

pre_mScreenStatus_df = preprocessing_mScreenStatus(mScreenStatus_df)

def cummean_mScreenStatus(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_mScreenStatus_df = cummean_mScreenStatus(pre_mScreenStatus_df)

def final_mScreenStatus(pre_mScreenStatus_df, cummean_mScreenStatus_df):
  merge_df = pre_mScreenStatus_df.merge(cummean_mScreenStatus_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_mScreenStatus_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'Q1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'Q2_Q3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_mScreenStatus_df = final_mScreenStatus(pre_mScreenStatus_df, cummean_mScreenStatus_df)

Q1@on_screen_ratio_cummean
Q1@on_screen_transitions_cummean
Q1@on_screen_entropy_cummean
Q1@avg_on_screen_cummean
Q1@std_on_screen_cummean
Q1@cov_on_screed_cummean
Q1@range_on_screen_cummean
Q1@mscreen_instability_cummean
Q2_Q3@on_screen_ratio_cummean
Q2_Q3@on_screen_transitions_cummean
Q2_Q3@on_screen_entropy_cummean
Q2_Q3@avg_on_screen_cummean
Q2_Q3@std_on_screen_cummean
Q2_Q3@cov_on_screed_cummean
Q2_Q3@range_on_screen_cummean
Q2_Q3@mscreen_instability_cummean


In [33]:
final_mScreenStatus_df

,subject_id,date,Q1@on_screen_ratio,Q1@on_screen_transitions,Q1@on_screen_entropy,Q1@avg_on_screen,Q1@std_on_screen,Q1@cov_on_screed,Q1@range_on_screen,Q1@mscreen_instability,Q2_Q3@on_screen_ratio,Q2_Q3@on_screen_transitions,Q2_Q3@on_screen_entropy,Q2_Q3@avg_on_screen,Q2_Q3@std_on_screen,Q2_Q3@cov_on_screed,Q2_Q3@range_on_screen,Q2_Q3@mscreen_instability
0,id01,2024-06-26,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1
1,id01,2024-06-27,1,0,1,1,1,1,1,1,0,0,0,0,0,1,0,0
2,id01,2024-06-28,0,0,0,1,1,1,1,1,0,0,0,0,1,1,1,1
3,id01,2024-06-29,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,id01,2024-06-30,0,1,1,0,0,1,1,1,1,0,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,id10,2024-09-22,1,1,0,1,1,1,1,1,0,0,0,1,1,0,1,0
799,id10,2024-09-23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
800,id10,2024-09-24,1,1,1,0,1,1,1,1,1,0,0,1,1,1,1,1
801,id10,2024-09-25,1,0,1,0,1,1,1,1,1,0,0,1,1,1,1,1


### mUsageStats

In [34]:
def preprocessing_mUsageStats(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': subj, 'date': date}
      for part, part_group in group.groupby('time_part'):
        all_labels_scores = []

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          total_time_sum = sum(item.get('total_time', 0) for item in flat if isinstance(item, dict))
          for item in flat:
            label = item['app_name']
            score = float(item['total_time'])
            if total_time_sum > 0:
              normalized_score = score / total_time_sum
              all_labels_scores.append((label, normalized_score))

          if all_labels_scores:
            label_list = [label for label, _ in all_labels_scores]
            score_list = [score for _, score in all_labels_scores]
            labels, scores = zip(*all_labels_scores)
            scores = list(scores)
            max_idx = np.argmax(scores)
            min_idx = np.argmin(scores)

            # 사용 최대 라벨
            # row[f'{part}@usagestats_max_label'] = labels[max_idx]

            # 사용 최소 라벨
            # row[f'{part}@usagestats_min_label'] = labels[min_idx]

            # 사용 범위
            range = scores[max_idx] - scores[min_idx]
            row[f'{part}@usagestats_range'] = range

            # 사용 고유값
            usagestats_cnt = len(set(label_list)) / len(label_list)
            row[f'{part}@usagestats_cnt'] = usagestats_cnt

            # 사용 형태 다양성 분포
            probs_counter = Counter(label_list)
            total = sum(probs_counter.values())
            probs = [v / total for v in probs_counter.values()]
            ent = entropy(probs)
            row[f'{part}@usagestats_entropy'] = ent

            # 기초 통계량
            mean = np.mean(scores) if sum(scores) > 0 else 0
            row[f'{part}@usagestats_mean'] = mean
            std = np.std(scores) if sum(scores) > 0 else 0
            row[f'{part}@usagestats_std'] = std
            cov = (np.std(scores) / np.mean(scores)) if sum(scores) > 0 else 0
            row[f'{part}@usagestats_cov'] = cov

            # 사용 불안전성
            row[f'{part}@usagestats_instability'] = (1 + ent) * (1 + usagestats_cnt) * (1 + cov)

      results.append(row)

    return pd.DataFrame(results)

pre_mUsageStats_df = preprocessing_mUsageStats(mUsageStats_df)

def cummean_mUsageStats(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_mUsageStats_df = cummean_mUsageStats(pre_mUsageStats_df)

def final_mUsageStats(pre_mUsageStats_df, cummean_mUsageStats_df):
  merge_df = pre_mUsageStats_df.merge(cummean_mUsageStats_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_mUsageStats_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'Q1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'Q2_Q3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_mUsageStats_df = final_mUsageStats(pre_mUsageStats_df, cummean_mUsageStats_df)

Q1@usagestats_range_cummean
Q1@usagestats_cnt_cummean
Q1@usagestats_entropy_cummean
Q1@usagestats_mean_cummean
Q1@usagestats_std_cummean
Q1@usagestats_cov_cummean
Q1@usagestats_instability_cummean
Q2_Q3@usagestats_range_cummean
Q2_Q3@usagestats_cnt_cummean
Q2_Q3@usagestats_entropy_cummean
Q2_Q3@usagestats_mean_cummean
Q2_Q3@usagestats_std_cummean
Q2_Q3@usagestats_cov_cummean
Q2_Q3@usagestats_instability_cummean


In [35]:
final_mUsageStats_df

,subject_id,date,Q1@usagestats_range,Q1@usagestats_cnt,Q1@usagestats_entropy,Q1@usagestats_mean,Q1@usagestats_std,Q1@usagestats_cov,Q1@usagestats_instability,Q2_Q3@usagestats_range,Q2_Q3@usagestats_cnt,Q2_Q3@usagestats_entropy,Q2_Q3@usagestats_mean,Q2_Q3@usagestats_std,Q2_Q3@usagestats_cov,Q2_Q3@usagestats_instability
0,id01,2024-06-26,1,0,1,0,1,0,0,1,0,1,0,1,1,1
1,id01,2024-06-27,1,0,1,0,1,1,1,1,0,0,1,1,1,0
2,id01,2024-06-28,1,0,0,1,1,0,0,1,1,0,1,1,1,1
3,id01,2024-06-29,0,1,0,0,0,0,0,1,0,0,1,0,0,0
4,id01,2024-06-30,1,0,1,0,0,1,1,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,id10,2024-09-22,1,0,0,1,1,0,0,0,1,1,1,1,1,1
786,id10,2024-09-23,0,1,0,0,0,0,1,0,0,0,0,0,0,0
787,id10,2024-09-24,1,0,1,0,0,1,1,0,1,0,1,1,1,1
788,id10,2024-09-25,1,0,1,0,1,1,1,1,1,1,0,0,1,1


### mwifi

In [36]:
def preprocessing_mwifi(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):

        rssi_list = []
        bssid_list = []

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          for device in flat:

              # rssi 목록
              rssi = int(device.get('rssi'))
              rssi_list.append(rssi)

              # bssid 목록
              bssid = str(device.get('bssid'))
              bssid_list.append(bssid)

        # rssi 고유값
        rssi_cnt = len(set(rssi_list)) / len(rssi_list)
        row[f'{part}@rssi_cnt'] = rssi_cnt

        # rssi 다양성
        rssi_counter = Counter(rssi_list)
        total = sum(rssi_counter.values())
        probs = [v / total for v in rssi_counter.values()]
        ent_rssi = entropy(probs)
        row[f'{part}@rssi_entropy'] = ent_rssi

        # bssid 고유값
        bssid_cnt = len(set(bssid_list)) / len(bssid_list)
        row[f'{part}@bssid_cnt'] = bssid_cnt

        # bssid 다양성
        bssid_counter = Counter(bssid_list)
        total = sum(bssid_counter.values())
        probs = [v / total for v in bssid_counter.values()]
        ent_bssid = entropy(probs)
        row[f'{part}@bssid_entropy'] = ent_bssid

        # 알수없는 rssi 비율
        weak_rssi = [r for r in rssi_list if r < -100 or r > - 25]
        rssi_Uncommon_cnt = len(weak_rssi)
        rssi_Uncommon_ratio =  rssi_Uncommon_cnt / len(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_Uncommon_ratio'] = rssi_Uncommon_ratio

        # 기초 통계
        mean = np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_mean'] = np.mean(rssi_list) if rssi_list else 0
        min = np.min(rssi_list) if rssi_list else 0
        max = np.max(rssi_list) if rssi_list else 0
        range = np.max(rssi_list) - np.min(rssi_list)
        row[f'{part}@rssi_range'] = range
        std = np.std(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_std'] = std
        cov = np.std(rssi_list) / np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_cov'] = cov

        # mble 불안전성
        row[f'{part}@wifi_instability'] = ((1 + ent_rssi) * (1 + rssi_cnt) *
         (1 + ent_bssid) * (1 + bssid_cnt) *
           (1 + cov))

      results.append(row)

    return pd.DataFrame(results)

pre_mWifi_df = preprocessing_mwifi(mWifi_df)

def cummean_mWifi(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_mWifi_df = cummean_mWifi(pre_mWifi_df)

def final_mWifi(pre_mWifi_df, cummean_mWifi_df):
  merge_df = pre_mWifi_df.merge(cummean_mWifi_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_mWifi_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'Q1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'Q2_Q3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_mWifi_df = final_mWifi(pre_mWifi_df, cummean_mWifi_df)

Q1@rssi_cnt_cummean
Q1@rssi_entropy_cummean
Q1@bssid_cnt_cummean
Q1@bssid_entropy_cummean
Q1@rssi_Uncommon_ratio_cummean
Q1@rssi_mean_cummean
Q1@rssi_range_cummean
Q1@rssi_std_cummean
Q1@rssi_cov_cummean
Q1@wifi_instability_cummean
Q2_Q3@rssi_cnt_cummean
Q2_Q3@rssi_entropy_cummean
Q2_Q3@bssid_cnt_cummean
Q2_Q3@bssid_entropy_cummean
Q2_Q3@rssi_Uncommon_ratio_cummean
Q2_Q3@rssi_mean_cummean
Q2_Q3@rssi_range_cummean
Q2_Q3@rssi_std_cummean
Q2_Q3@rssi_cov_cummean
Q2_Q3@wifi_instability_cummean


In [37]:
final_mWifi_df

,subject_id,date,Q1@rssi_cnt,Q1@rssi_entropy,Q1@bssid_cnt,Q1@bssid_entropy,Q1@rssi_Uncommon_ratio,Q1@rssi_mean,Q1@rssi_range,Q1@rssi_std,Q1@rssi_cov,Q1@wifi_instability,Q2_Q3@rssi_cnt,Q2_Q3@rssi_entropy,Q2_Q3@bssid_cnt,Q2_Q3@bssid_entropy,Q2_Q3@rssi_Uncommon_ratio,Q2_Q3@rssi_mean,Q2_Q3@rssi_range,Q2_Q3@rssi_std,Q2_Q3@rssi_cov,Q2_Q3@wifi_instability
0,id01,2024-06-26,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0,1,0,0
1,id01,2024-06-27,0,1,1,1,0,0,1,0,1,1,0,1,0,1,1,1,0,0,1,1
2,id01,2024-06-28,0,1,0,1,0,1,1,1,0,0,0,0,0,1,1,0,0,0,1,0
3,id01,2024-06-29,1,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,1
4,id01,2024-06-30,0,1,1,1,1,0,1,0,1,1,0,0,1,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780,id10,2024-09-22,1,0,1,0,0,0,0,0,1,1,0,1,0,0,0,1,0,1,0,0
781,id10,2024-09-23,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
782,id10,2024-09-24,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0
783,id10,2024-09-25,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


### wHr

In [38]:
def preprocessing_wHr(dfs):
  df = sleep_lifelog(dfs)
  df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
  df['date'] = df['lifelog_date'].dt.date
  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df = df.sort_values(['subject_id', 'timestamp'])
  df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
  col = find_col(df)

  results = []

  for (sub, date), group in df.groupby(['subject_id', 'date']):
    row = {'subject_id':sub, 'date': date}

    for part, part_group in group.groupby('time_part'):
      hr_list = []
      for entry in part_group[col]:
        flat = ast.literal_eval(entry) if isinstance(entry, str) else entry
        hr_list.extend([int(hr) for hr in flat if hr is not None])

      # 심장박동 기초 통계량
      mean = np.mean(hr_list) if hr_list else 0
      row[f'{part}@hr_mean'] = mean
      std = np.std(hr_list) if hr_list else 0
      row[f'{part}@hr_std'] = std
      max = np.max(hr_list) if hr_list else 0
      # row[f'{part}@hr_max'] = max
      min = np.min(hr_list) if hr_list else 0
      # row[f'{part}@hr_min'] = min
      range = max - min
      row[f'{part}@hr_range'] = range
      cov = (np.std(hr_list) / np.mean(hr_list)) if hr_list else 0
      row[f'{part}@hr_cov'] = cov

      # 비정상적인 박동
      hr_over = len([hr for hr in hr_list if hr >= 100]) / len(hr_list)
      # row[f'{part}@hr_100_over_cnt'] = hr_over
      hr_under = len([hr for hr in hr_list if hr <= 60]) / len(hr_list)
      # row[f'{part}@hr_60_under_cnt'] = hr_under
      hr_abnormal = hr_over + hr_under
      row[f'{part}@hr_abnormal_cnt'] = hr_abnormal

      # 심장박동 다양성 분포
      hist, _ = np.histogram(hr_list, bins=12, range=(40, 160), density=True)
      ent = entropy(hist)
      row[f'{part}@hr_entropy'] = ent

      # 심장박동 불안전성
      row[f'{part}@hr_instability'] = (1 + ent) * (1 + cov) * (1 + hr_abnormal)

    results.append(row)

  return pd.DataFrame(results)

pre_wHr_df = preprocessing_wHr(wHr_df)

def cummean_wHr(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_wHr_df = cummean_wHr(pre_wHr_df)

def final_wHr(pre_wHr_df, cummean_wHr_df):
  merge_df = pre_wHr_df.merge(cummean_wHr_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_wHr_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'Q1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'Q2_Q3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_wHr_df = final_wHr(pre_wHr_df, cummean_wHr_df)

Q1@hr_mean_cummean
Q1@hr_std_cummean
Q1@hr_range_cummean
Q1@hr_cov_cummean
Q1@hr_abnormal_cnt_cummean
Q1@hr_entropy_cummean
Q1@hr_instability_cummean
Q2_Q3@hr_mean_cummean
Q2_Q3@hr_std_cummean
Q2_Q3@hr_range_cummean
Q2_Q3@hr_cov_cummean
Q2_Q3@hr_abnormal_cnt_cummean
Q2_Q3@hr_entropy_cummean
Q2_Q3@hr_instability_cummean


In [39]:
final_wHr_df

,subject_id,date,Q1@hr_mean,Q1@hr_std,Q1@hr_range,Q1@hr_cov,Q1@hr_abnormal_cnt,Q1@hr_entropy,Q1@hr_instability,Q2_Q3@hr_mean,Q2_Q3@hr_std,Q2_Q3@hr_range,Q2_Q3@hr_cov,Q2_Q3@hr_abnormal_cnt,Q2_Q3@hr_entropy,Q2_Q3@hr_instability
0,id01,2024-06-26,0,1,1,1,0,1,0,1,1,0,1,1,1,1
1,id01,2024-06-27,1,0,1,0,0,1,0,1,1,1,1,1,1,1
2,id01,2024-06-28,0,0,0,0,0,0,0,1,1,0,1,1,1,1
3,id01,2024-06-29,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,id01,2024-06-30,0,0,0,0,0,0,0,0,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,id10,2024-09-22,0,0,0,0,0,0,0,1,0,0,0,0,0,0
682,id10,2024-09-23,1,1,1,1,1,1,1,0,0,0,0,0,0,0
683,id10,2024-09-24,0,0,0,0,0,0,0,1,1,0,0,1,0,1
684,id10,2024-09-25,0,0,0,0,0,0,0,1,0,1,0,1,0,0


### wLight

In [40]:
def preprocessing_wLight(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (sub, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': sub, 'date': date}
      for time_label, block_group in group.groupby('time_part'):
        light_list = block_group[col].tolist()

        # wLight 기초 통계
        mean = np.mean(light_list)
        row[f'{time_label}@wLight_mean'] = mean
        std = np.std(light_list)
        row[f'{time_label}@wLight_std'] = std
        cov = np.std(light_list) / np.mean(light_list)
        row[f'{time_label}@wLight_cov'] = cov
        max = np.max(light_list)
        # row[f'{time_label}@wLight_max'] = max
        min = np.min(light_list)
        # row[f'{time_label}@wLight_min'] = min
        range = max - min
        row[f'{time_label}@wLight_range'] = range

        # might 고유값
        light_cnt = len(set(light_list)) / len(light_list)
        row[f'{time_label}@wLight_cnt'] = light_cnt

        # wLight 다양성
        hist_light, _ = np.histogram(light_list, bins=10, density=True)
        ent = entropy(hist_light) if hist_light.sum() > 0 else 0
        row[f'{time_label}@wLight_entropy'] = ent

        # wLight 불안전성
        row[f'{time_label}@wLight_instability'] = (1 + ent) * (1 + cov) * (1 + light_cnt)

      results.append(row)

    return pd.DataFrame(results)

pre_wLight_df = preprocessing_wLight(wLight_df)

def cummean_wLight(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_wLight_df = cummean_wLight(pre_wLight_df)

def final_wLight(pre_wLight_df, cummean_wLight_df):
  merge_df = pre_wLight_df.merge(cummean_wLight_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_wLight_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'Q1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'Q2_Q3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_wLight_df = final_wLight(pre_wLight_df, cummean_wLight_df)

Q1@wLight_mean_cummean
Q1@wLight_std_cummean
Q1@wLight_cov_cummean
Q1@wLight_range_cummean
Q1@wLight_cnt_cummean
Q1@wLight_entropy_cummean
Q1@wLight_instability_cummean
Q2_Q3@wLight_mean_cummean
Q2_Q3@wLight_std_cummean
Q2_Q3@wLight_cov_cummean
Q2_Q3@wLight_range_cummean
Q2_Q3@wLight_cnt_cummean
Q2_Q3@wLight_entropy_cummean
Q2_Q3@wLight_instability_cummean


In [41]:
final_wLight_df

,subject_id,date,Q1@wLight_mean,Q1@wLight_std,Q1@wLight_cov,Q1@wLight_range,Q1@wLight_cnt,Q1@wLight_entropy,Q1@wLight_instability,Q2_Q3@wLight_mean,Q2_Q3@wLight_std,Q2_Q3@wLight_cov,Q2_Q3@wLight_range,Q2_Q3@wLight_cnt,Q2_Q3@wLight_entropy,Q2_Q3@wLight_instability
0,id01,2024-06-26,0,1,1,1,1,0,1,0,0,1,0,0,0,1
1,id01,2024-06-27,0,1,1,1,1,0,1,1,0,0,0,1,1,1
2,id01,2024-06-28,0,0,0,0,0,1,0,0,0,1,0,1,1,1
3,id01,2024-06-29,0,0,0,0,0,0,0,1,1,0,0,1,1,0
4,id01,2024-06-30,0,0,0,0,0,1,0,1,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,id10,2024-09-22,0,0,0,0,1,1,0,1,1,0,0,1,1,1
746,id10,2024-09-23,0,0,0,0,0,1,0,0,0,0,0,0,0,0
747,id10,2024-09-24,0,0,1,0,0,0,1,1,0,1,0,0,1,1
748,id10,2024-09-25,0,0,0,0,0,1,0,0,0,1,0,0,0,1


### wPedo

In [42]:
def preprocessing_wPedo(dfs):
  df = sleep_lifelog(dfs)
  df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
  df['date'] = df['lifelog_date'].dt.date
  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df = df.sort_values(['subject_id', 'timestamp'])
  df['time_part'] = df['timestamp'].dt.hour.map(cut_time)

  results = []

  for (subj, date), group in df.groupby(['subject_id', 'date']):
    row = {'subject_id':subj, 'date':date}
    for part, part_group in group.groupby('time_part'):
      step_list = []
      step_frequency_list = []
      distance_list = []
      speed_list = []
      burned_calories_list = []

      for _, entry in part_group.iterrows():
        step_list.append(entry['step'])
        step_frequency_list.append(entry['step_frequency'])
        distance_list.append(entry['distance'])
        speed_list.append(entry['speed'])
        burned_calories_list.append(entry['burned_calories'])

      # step 기초통계량
      step_sum = (np.sum(step_list) / len(step_list)) if sum(step_list) > 0 else 0
      row[f'{part}@step_sum'] = step_sum
      step_mean = np.mean(step_list) if sum(step_list) > 0 else 0
      row[f'{part}@step_mean'] = step_mean
      step_std = np.std(step_list) if sum(step_list) > 0 else 0
      row[f'{part}@step_std'] = step_std
      step_cov = (np.std(step_list) / np.mean(step_list)) if sum(step_list) > 0 else 0
      row[f'{part}@step_cov'] = step_cov

      # step 전이 수
      step_arr = np.array(step_list)
      transitions = np.sum(step_arr[1:] != step_arr[:-1]) / len(step_arr)
      row[f'{part}@step_transition'] = transitions

      # step 다양성
      hist, _ = np.histogram(step_arr, bins=10, density=True)
      step_entropy = entropy(hist) if hist.sum() > 0 else 0
      row[f'{part}@step_entropy'] = step_entropy

      # 보속 평균
      sf_mean = np.mean(step_frequency_list) if np.sum(step_frequency_list) > 0 else 0
      row[f'{part}@step_frequency_mean'] = sf_mean

      # 총 이동거리
      d_sum = np.sum(distance_list) / len(distance_list)
      row[f'{part}@distance_sum'] = d_sum

      # 속도 기초통계량
      speed_mean = np.mean(speed_list) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_mean'] = speed_mean
      speed_std = np.std(speed_list) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_std'] = speed_std
      speed_cov = (np.std(speed_list) / np.mean(speed_list)) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_cov'] = speed_std
      speed_max = np.max(speed_list) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_max'] = speed_max

      # 총 소비된 칼로리
      bc_sum = (np.sum(burned_calories_list) / len(burned_calories_list)) if burned_calories_list and np.sum(burned_calories_list) > 0 else 0
      row[f'{part}@burned_calories_sum'] = bc_sum

      # 추정 보폭 평균
      if step_list and distance_list:
        step_sum = np.sum(step_list)
        if step_sum > 0:
          stride_mean = np.sum(distance_list) / step_sum
        else:
          stride_mean = 0
      else:
          stride_mean = 0
      row[f'{part}@stride_mean'] = stride_mean

      # pedo 불안전성
      instability = ((1 + step_entropy) * (1 + step_cov) *
       (1 + transitions) *
        (1 + speed_cov) * (1 + stride_mean))
      row[f'{part}@pedo_instability'] = instability

    results.append(row)

  return pd.DataFrame(results)

pre_wPedo_df = preprocessing_wPedo(wPedo_df)

def cummean_wPedo(df):
    mean_df = pd.DataFrame()

    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
      mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

    return mean_df.reset_index()

cummean_wPedo_df = cummean_wPedo(pre_wPedo_df)

def final_wPedo(pre_wPedo_df, cummean_wPedo_df):
  merge_df = pre_wPedo_df.merge(cummean_wPedo_df, on='subject_id', suffixes=('', '_cummean'))

  final_df = pd.DataFrame()
  final_df['subject_id'] = merge_df['subject_id']
  final_df['date'] = merge_df['date']

  for col in pre_wPedo_df.columns:
    if '@' in col:
      cummean_col = f'{col}_cummean'
      print(cummean_col)
      if 'Q1' in col:
        final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
      elif 'Q2_Q3' in col:
        final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

  return final_df

final_wPedo_df = final_wPedo(pre_wPedo_df, cummean_wPedo_df)

Q1@step_sum_cummean
Q1@step_mean_cummean
Q1@step_std_cummean
Q1@step_cov_cummean
Q1@step_transition_cummean
Q1@step_entropy_cummean
Q1@step_frequency_mean_cummean
Q1@distance_sum_cummean
Q1@pedo_speed_mean_cummean
Q1@pedo_speed_std_cummean
Q1@pedo_speed_cov_cummean
Q1@pedo_speed_max_cummean
Q1@burned_calories_sum_cummean
Q1@stride_mean_cummean
Q1@pedo_instability_cummean
Q2_Q3@step_sum_cummean
Q2_Q3@step_mean_cummean
Q2_Q3@step_std_cummean
Q2_Q3@step_cov_cummean
Q2_Q3@step_transition_cummean
Q2_Q3@step_entropy_cummean
Q2_Q3@step_frequency_mean_cummean
Q2_Q3@distance_sum_cummean
Q2_Q3@pedo_speed_mean_cummean
Q2_Q3@pedo_speed_std_cummean
Q2_Q3@pedo_speed_cov_cummean
Q2_Q3@pedo_speed_max_cummean
Q2_Q3@burned_calories_sum_cummean
Q2_Q3@stride_mean_cummean
Q2_Q3@pedo_instability_cummean


In [43]:
final_wPedo_df

,subject_id,date,Q1@step_sum,Q1@step_mean,Q1@step_std,Q1@step_cov,Q1@step_transition,Q1@step_entropy,Q1@step_frequency_mean,Q1@distance_sum,Q1@pedo_speed_mean,Q1@pedo_speed_std,Q1@pedo_speed_cov,Q1@pedo_speed_max,Q1@burned_calories_sum,Q1@stride_mean,Q1@pedo_instability,Q2_Q3@step_sum,Q2_Q3@step_mean,Q2_Q3@step_std,Q2_Q3@step_cov,Q2_Q3@step_transition,Q2_Q3@step_entropy,Q2_Q3@step_frequency_mean,Q2_Q3@distance_sum,Q2_Q3@pedo_speed_mean,Q2_Q3@pedo_speed_std,Q2_Q3@pedo_speed_cov,Q2_Q3@pedo_speed_max,Q2_Q3@burned_calories_sum,Q2_Q3@stride_mean,Q2_Q3@pedo_instability
0,id01,2024-06-26,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
1,id01,2024-06-27,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,0,0,0
2,id01,2024-06-28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,id01,2024-06-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,id01,2024-06-30,0,0,1,1,0,0,0,0,0,1,1,1,0,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,id10,2024-09-21,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,0,0,0,1,1,1
730,id10,2024-09-22,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
731,id10,2024-09-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
732,id10,2024-09-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1


# 데이터 합치기

In [44]:
dfs_list = []
dfs = dataframe.copy()
for k, v in dfs.items():
  pre_name = f'pre_{k}_df'
  dfs_list.append(pre_name)
  print(pre_name)

# pre_list = {'pre_' + k + '_df2': v for k, v in dfs.items()}
# pre_name1

pre_mACStatus_df
pre_mActivity_df
pre_mAmbience_df
pre_mBle_df
pre_mGps_df
pre_mLight_df
pre_mScreenStatus_df
pre_mUsageStats_df
pre_mWifi_df
pre_wHr_df
pre_wLight_df
pre_wPedo_df


In [45]:
df_list = [final_mACStatus_df,
final_mActivity_df,
final_mAmbience_df,
final_mBle_df,
final_mGps_df,
final_mLight_df,
final_mScreenStatus_df,
final_mUsageStats_df,
final_mWifi_df,
final_wHr_df,
final_wLight_df,
final_wPedo_df]

In [46]:
prep_df = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'date'], how='outer'), df_list)

In [47]:
# 전처리된 데이터 저장
prep_df.to_csv('/content/drive/MyDrive/1데이콘/ETRI_휴먼이해_인공지능/ETRI_lifelog_dataset/prep_df_Q1-3.csv',index=False)